Code to do gram schmidt in several variables in the Drury-Arveson space. (Just realized I should've talked about this today; I'll write up some notes and post in Teams tonight.)

In [1]:
#setup functions: list of monomials, extracting coefficients
Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex') #define the ring!!!
def buildmonoms(whichOG): #puts the monomials in degree lexicographic order
    Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex')
    monomslist=[]
    m=0
    for totaldeg in range(0,whichOG+1):
        k=0;l=totaldeg;
        while k<=totaldeg and m<=whichOG:
            monomslist.append(sage_eval('z1^'+str(l)+'*z2^'+str(k),locals=vars()))#so it's not just strings
            #print (k,l),newcoefs[(k, l)]
            m=m+1
            k=k+1
            l=l-1
    return monomslist
def multideg(monom): #take a monomial in z1 and z2 and return an ordered pair with its multidegree 
                    #(there's probably a way to do this more generally)
    z1deg=monom.degree(z1)
    z2deg=monom.degree(z2)
    return (z1deg,z2deg)
def coeffdict(poly,monomslist): #takes a polynomial (as an expression) and a pregenerated list of monomials and returns a dictionary
                                # keys are ordered pairs of degrees of a monomial, entries are coefficients
    polydict={}
    for monom in monomslist:
        multi=multideg(monom)
        #print(multi)
        d={z1:multi[0],z2:multi[1]}
        #print(d)
        polydict.update({multideg(monom):poly.coefficient(d)})
    return polydict

In [3]:
#inner product
def DAIP(f,g,monomslist):
    cfsf=coeffdict(f,monomslist)#the coefficients paired into a dictionary with the multi-indices
    cfsg=coeffdict(g,monomslist)
    #print(cfsf)
    #print(cfsg)
    pairs=list(set(cfsf.keys()).union(set(cfsg.keys()))) #pretty sure unioning them is unnecessary, but *shrug*
    #print('pairs')
    #print(pairs)
    m=0; ip=0;
    for (k,l) in pairs:
        #print("");print("__________");print("")
        #print(k,l)
        plus=(factorial(k+l))
        #print("plus",plus)
        times=(factorial(k)*factorial(l))
        #print("times",times)
        #print(times/plus)
        wratio=QQ(times/plus)
        #print(wratio)
        term=wratio*(cfsf[pairs[m]])*(cfsg[pairs[m]])#conjugate. we'll have to find a workaround if we ever do complex coeffs
        #print pairs[m],cfsg[pairs[m]]
        #print "term", term
        ip=ip+term
        #print(ip)
        m=m+1
    return ip
#
#
def DAwIP(f,g,weight,monomslist):
    wip=DAIP(f*weight,g*weight,monomslist)
    return wip
def DAwproj(f,g,weight,monomslist):
    ip=DAwIP(f,g,weight,monomslist)
    normsquared=DAwIP(g,g,weight,monomslist)
    proj=(ip/normsquared)*g
    return proj

In [2]:
#Gram Schmidt
#given an ordered list of linearly independent basis vectors (monomials in this case),
#and a list of orthogonal polynomials previously computed from the given basis vectors,
# this returns the next one. IT DOES NOT CHANGE THE INPUTTED LISTS. (could probably write a recursive function that would be more efficient, but I'm lazy)
def DAnextGS(basislist,OGlist,weight,monomslist):
    error=0
    nextbasis=basislist[len(OGlist)]
    #print "nextbasis=",nextbasis
    for phi in OGlist:
        error=error+DAwproj(nextbasis,phi,weight,monomslist)
        #print "error",error
    PHInext=nextbasis-error
    return PHInext
def DA_GS_UpTo(basislist,weight,howmany,monomslist):
    OGlist=[1]
    for n in range(1,howmany):
        OGlist=OGlist+[DAnextGS(basislist,OGlist,weight,monomslist)]
    return OGlist

In [5]:
numofmons=40
monoms=buildmonoms(numofmons)
print(monoms)
S.<sqrt2> = QQ.extension(x^2 - 2)
g=1-1/sqrt2*(z1+z2)
print('   ')
show(g)
higherpowerg=g^3
show(higherpowerg)

[1, z1, z2, z1^2, z1*z2, z2^2, z1^3, z1^2*z2, z1*z2^2, z2^3, z1^4, z1^3*z2, z1^2*z2^2, z1*z2^3, z2^4, z1^5, z1^4*z2, z1^3*z2^2, z1^2*z2^3, z1*z2^4, z2^5, z1^6, z1^5*z2, z1^4*z2^2, z1^3*z2^3, z1^2*z2^4, z1*z2^5, z2^6, z1^7, z1^6*z2, z1^5*z2^2, z1^4*z2^3, z1^3*z2^4, z1^2*z2^5, z1*z2^6, z2^7, z1^8, z1^7*z2, z1^6*z2^2, z1^5*z2^3, z1^4*z2^4]
   


(-1/2*sqrt2)*z1 + (-1/2*sqrt2)*z2 + 1

(-1/4*sqrt2)*z1^3 + (-3/4*sqrt2)*z1^2*z2 + (-3/4*sqrt2)*z1*z2^2 + (-1/4*sqrt2)*z2^3 + 3/2*z1^2 + 3*z1*z2 + 3/2*z2^2 + (-3/2*sqrt2)*z1 + (-3/2*sqrt2)*z2 + 1

In [8]:
DAOGs=DA_GS_UpTo(monoms,higherpowerg,30,monoms)
for OG in DAOGs:
    show(OG)

1

z1 + (3/8*sqrt2)

z2 + (3/8*sqrt2)

z1^2 + (3/5*sqrt2)*z1 + 3/10

z1*z2 + (3/10*sqrt2)*z1 + (3/10*sqrt2)*z2 + 3/10

...

496449732849931/1552255546482953461*z1^6 + 10534370439586754/1552255546482953461*z1^5*z2 - 14058146011012812/1552255546482953461*z1^4*z2^2 + 372220277826008582/1552255546482953461*z1^3*z2^3 + z1^2*z2^4 + (6756534418343170/4656766639448860383*sqrt2)*z1^5 + (12277780087954073/4656766639448860383*sqrt2)*z1^4*z2 + (131683227515436753/1552255546482953461*sqrt2)*z1^3*z2^2 + (1049061790788908101/1552255546482953461*sqrt2)*z1^2*z2^3 + (404691193411176815/1552255546482953461*sqrt2)*z1*z2^4 + (-9983523249732549/1552255546482953461*sqrt2)*z2^5 + 46060452179669923/18627066557795441532*z1^4 + 176011510818439955/4656766639448860383*z1^3*z2 + 3216824765158598321/6209022185931813844*z1^2*z2^2 + 4979322497915264431/9313533278897720766*z1*z2^3 + 1640706647979673169/37254133115590883064*z2^4 + (224219218525009255/43463155301522696908*sqrt2)*z1^3 + (5555087133819355649/43463155301522696908*sqrt2)*z1^2*z2 + (12738825113783480917/43463155301522696908*sqrt2)*z1*z2^2 + (3316479019970321329/43463155301522696908*sqrt2)*z2^3 + 4440521226083230945/130389465904568090724*z1^2 + 13063227767945761555/65194732952284045362*z1*z2 + 2668919007162898195/21731577650761348454*z2^2 + (224930998170682543/6209022185931813844*sqrt2)*z1 + (743762848195097155/12418044371863627688*sqrt2)*z2 + 1542978215786354743/49672177487454510752

1104974127107167/18652136515252844512*z1^6 + 35022062373683373/9326068257626422256*z1^5*z2 - 126794653275381367/4663034128813211128*z1^4*z2^2 + 1582150511855218039/9326068257626422256*z1^3*z2^3 + 16492144297997921505/18652136515252844512*z1^2*z2^4 + z1*z2^5 + (6389497459167479/9326068257626422256*sqrt2)*z1^5 + (-332068301233108603/55956409545758533536*sqrt2)*z1^4*z2 + (619553283609061229/18652136515252844512*sqrt2)*z1^3*z2^2 + (9276298598045222865/18652136515252844512*sqrt2)*z1^2*z2^3 + (17108131204547913683/18652136515252844512*sqrt2)*z1*z2^4 + (2198219537930442191/18652136515252844512*sqrt2)*z2^5 - 140383377458084233/223825638183034134144*z1^4 + 128903009546628367/55956409545758533536*z1^3*z2 + 23799916246982228269/74608546061011378048*z1^2*z2^2 + 113728668880223944691/111912819091517067072*z1*z2^3 + 143140233173538977677/447651276366068268288*z2^4 + (-46427862966947427/74608546061011378048*sqrt2)*z1^3 + (33271847149544141245/522259822427079646336*sqrt2)*z1^2*z2 + (206577655006163791585/522259822427079646336*sqrt2)*z1*z2^2 + (128636894141136853709/522259822427079646336*sqrt2)*z2^3 + 968671885341460105/74608546061011378048*z1^2 + 165156621533566867735/783389733640619469504*z1*z2 + 68657641522693393575/261129911213539823168*z2^2 + (2296068395852956555/74608546061011378048*sqrt2)*z1 + (14491286665112069255/149217092122022756096*sqrt2)*z2 + 834205379932053975/20581667878899690496

161932965798229/40125493311456602283*z1^6 + 44916997311495636/13375164437152200761*z1^5*z2 - 1252828292181000727/40125493311456602283*z1^4*z2^2 + 5775466097519907584/40125493311456602283*z1^3*z2^3 + 13901038107086825309/13375164437152200761*z1^2*z2^4 + 60668882565359697028/40125493311456602283*z1*z2^5 + z2^6 + (22620431621546047/40125493311456602283*sqrt2)*z1^5 + (-915950812344783457/120376479934369806849*sqrt2)*z1^4*z2 + (546213984241083974/40125493311456602283*sqrt2)*z1^3*z2^2 + (7110492516653768766/13375164437152200761*sqrt2)*z1^2*z2^3 + (48295531256920196747/40125493311456602283*sqrt2)*z1*z2^4 + (41292864706753324289/40125493311456602283*sqrt2)*z2^5 - 144161084505398188/120376479934369806849*z1^4 - 1226354988281809496/120376479934369806849*z1^3*z2 + 12334031682137854912/40125493311456602283*z1^2*z2^2 + 145620697829023761976/120376479934369806849*z1*z2^3 + 168238595787034171052/120376479934369806849*z2^4 + (-32815270075166580/13375164437152200761*sqrt2)*z1^3 + (2127951062190236164/40125493311456602283*sqrt2)*z1^2*z2 + (17457358270366127836/40125493311456602283*sqrt2)*z1*z2^2 + (29564369241380909180/40125493311456602283*sqrt2)*z2^3 + 25711503681468220/3086576408573584791*z1^2 + 25900122192189318080/120376479934369806849*z1*z2 + 8203015500755463260/13375164437152200761*z2^2 + (1161553410144002092/40125493311456602283*sqrt2)*z1 + (7597563330109751572/40125493311456602283*sqrt2)*z2 + 919705952384940728/13375164437152200761

z1^7 + (52743948987678170719/59119254851756782608*sqrt2)*z1^6 + (-49637843044382908/410550380914977657*sqrt2)*z1^5*z2 + (-132622590917494909/59119254851756782608*sqrt2)*z1^4*z2^2 + (13920553653066473/3694953428234798913*sqrt2)*z1^3*z2^3 + (-8837844405754945/19706418283918927536*sqrt2)*z1^2*z2^4 + (-1173888554459579/3694953428234798913*sqrt2)*z1*z2^5 + (-1538156976350869/19706418283918927536*sqrt2)*z2^6 + 154304507782490535071/147798137129391956520*z1^5 - 18002246086895341789/88678882277635173912*z1^4*z2 + 10624868763369931/14779813712939195652*z1^3*z2^2 + 1701645368199343/547400507886636876*z1^2*z2^3 - 20882435308344883/29559627425878391304*z1*z2^4 - 5861559189438589/29559627425878391304*z2^5 + (85363145317500473465/177357764555270347824*sqrt2)*z1^4 + (-17971646203919828483/177357764555270347824*sqrt2)*z1^3*z2 + (41770809975491765/29559627425878391304*sqrt2)*z1^2*z2^2 + (39623467868362889/88678882277635173912*sqrt2)*z1*z2^3 + (-8189699993438131/44339441138817586956*sqrt2)*z2^4 + 1198195865241649571/3284403047319821256*z1^3 - 1059934677543690061/14779813712939195652*z1^2*z2 + 29427615596891347/29559627425878391304*z1*z2^2 - 1493269354474369/29559627425878391304*z2^3 + (6634054798266762673/59119254851756782608*sqrt2)*z1^2 + (-3135838540585844725/177357764555270347824*sqrt2)*z1*z2 + (1089160207415425/9853209141959463768*sqrt2)*z2^2 + 6154987472005177435/118238509703513565216*z1 - 577822421006640509/118238509703513565216*z2 + (1412392015410255671/197064182839189275360*sqrt2)

6818877942275977429/35411393172978231691*z1^7 + z1^6*z2 + (79880632559064945727/283291145383825853528*sqrt2)*z1^6 + (25625094693112527114/35411393172978231691*sqrt2)*z1^5*z2 + (-24726703576334934229/283291145383825853528*sqrt2)*z1^4*z2^2 + (278252010701335894/35411393172978231691*sqrt2)*z1^3*z2^3 + (7874778025308520773/283291145383825853528*sqrt2)*z1^2*z2^4 + (28405701625096156/2083023127822248923*sqrt2)*z1*z2^5 + (580382828848308729/283291145383825853528*sqrt2)*z2^6 + 66625452706459659725/141645572691912926764*z1^5 + 275306831248046217905/424936718075738780292*z1^4*z2 - 7886512853871159485/70822786345956463382*z1^3*z2^2 + 2367389814357405741/70822786345956463382*z1^2*z2^3 + 258518724186266677/8332092511288995692*z1*z2^4 + 893934205003457819/141645572691912926764*z2^5 + (80562997634600669969/283291145383825853528*sqrt2)*z1^4 + (180523332815732996273/849873436151477560584*sqrt2)*z1^3*z2 + (-4519900741626675121/141645572691912926764*sqrt2)*z1^2*z2^2 + (8445716615589425825/424936718075738780292*sqrt2)*z1*z2^3 + (2742750403909741603/424936718075738780292*sqrt2)*z2^4 + 888259026180903635/3310580997807647704*z1^3 + 209897153754371387783/1983038017686780974696*z1^2*z2 - 3448853942785743371/991519008843390487348*z1*z2^2 + 9445956363571356617/991519008843390487348*z2^3 + (7053094308373500983/70822786345956463382*sqrt2)*z1^2 + (17474374762690384259/849873436151477560584*sqrt2)*z1*z2 + (21604679456498625/8332092511288995692*sqrt2)*z2^2 + 46958959832239793509/849873436151477560584*z1 + 5137157599476201121/849873436151477560584*z2 + (2642609982146136345/283291145383825853528*sqrt2)

In [0]:
#getting LaTeX output for the orthogonal polynomials (will need to do a find and replace to handle the sqrt2)
for j in range(len(DAOGs)):
    h=DAOGs[j]
    print('\\phi_{',j,'}&='+latex(h)+'\\\\')

In [0]:
#getting LaTeX output for the list of norms of the OGs
for m in range(0,len(DAOGs)):
    for n in range(0,len(DAOGs)):
        ort=DAIP(DAOGs[m]*g,DAOGs[n]*g,monoms)
        if ort!=0:
            print("\\norm{\\phi_{",m,"}}^2&=",ort,"\\\\")

In [0]:
#looking at reproducing kernels at zero:
#need to sum terms that look like
#1/norm^2 conj(phi(0)g(0)) phi(z1,z2)f(z1,z2)
#but conjugate doesn't matter I think
#
#
whichspace=2
RK=0
for j in range(whichspace+1):
    show(DAOGs[j])
    normsq=DAIP(DAOGs[j]*g,DAOGs[j]*g,monoms)
    show(normsq)
    piecej=g*DAOGs[j]
    show(piecej)
    newterm=1/normsq*piecej(0,0)*piecej
    show(newterm)
    RK=RK+newterm
    print("____________")
print("")
show(RK)

In [0]:
show(RK*6)

In [0]:
phi1g=p1(z1,z2)

In [9]:
print(DAOGs)

[1, z1 + (3/8*sqrt2), z2 + (3/8*sqrt2), z1^2 + (3/5*sqrt2)*z1 + 3/10, z1*z2 + (3/10*sqrt2)*z1 + (3/10*sqrt2)*z2 + 3/10, z2^2 + (3/5*sqrt2)*z2 + 3/10, z1^3 + (3/4*sqrt2)*z1^2 + 3/5*z1 + (1/8*sqrt2), z1^2*z2 + (1/4*sqrt2)*z1^2 + (1/2*sqrt2)*z1*z2 + 2/5*z1 + 1/5*z2 + (1/8*sqrt2), z1*z2^2 + (1/2*sqrt2)*z1*z2 + (1/4*sqrt2)*z2^2 + 1/5*z1 + 2/5*z2 + (1/8*sqrt2), z2^3 + (3/4*sqrt2)*z2^2 + 3/5*z2 + (1/8*sqrt2), z1^4 + (6/7*sqrt2)*z1^3 + 6/7*z1^2 + (2/7*sqrt2)*z1 + 3/28, z1^3*z2 + (3/14*sqrt2)*z1^3 + (9/14*sqrt2)*z1^2*z2 + 3/7*z1^2 + 3/7*z1*z2 + (3/14*sqrt2)*z1 + (1/14*sqrt2)*z2 + 3/28, z1^2*z2^2 + (3/7*sqrt2)*z1^2*z2 + (3/7*sqrt2)*z1*z2^2 + 1/7*z1^2 + 4/7*z1*z2 + 1/7*z2^2 + (1/7*sqrt2)*z1 + (1/7*sqrt2)*z2 + 3/28, z1*z2^3 + (9/14*sqrt2)*z1*z2^2 + (3/14*sqrt2)*z2^3 + 3/7*z1*z2 + 3/7*z2^2 + (1/14*sqrt2)*z1 + (3/14*sqrt2)*z2 + 3/28, z2^4 + (6/7*sqrt2)*z2^3 + 6/7*z2^2 + (2/7*sqrt2)*z2 + 3/28, z1^5 + (15/16*sqrt2)*z1^4 + 15/14*z1^3 + (25/56*sqrt2)*z1^2 + 15/56*z1 + (3/64*sqrt2), z1^4*z2 + (3/16*sqrt2